In [1]:
%pwd

'/home/ajith/projects/Spaceship_Titanic_MLOps_Project/research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'/home/ajith/projects/Spaceship_Titanic_MLOps_Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    local_data_file: Path
    cloud_config_zipfile: Path
    authentication_token: Path

In [5]:

from titanicSpaceShip.utils.common import read_yaml, create_directories

In [10]:
import os 
import urllib.request as request
import zipfile
from titanicSpaceShip import logger
from titanicSpaceShip.utils.common import get_size

In [6]:
import time
import csv
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        delay = 5
        max_retries = 1
        for _ in range(max_retries):
            try:
                if not os.path.exists(self.config.local_data_file):
                    # This secure connect bundle is autogenerated when you download your SCB, 
                    # if yours is different update the file name below
                    cloud_config= {
                    'secure_connect_bundle': self.config.cloud_config_zipfile
                    }
                    # This token JSON file is autogenerated when you download your token, 
                    # if yours is different update the file name below
                    with open(self.config.authentication_token) as f:
                        secrets = json.load(f)
                    CLIENT_ID = secrets["clientId"]
                    CLIENT_SECRET = secrets["secret"]

                    auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
                    cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
                    session = cluster.connect()

                    row = session.execute('SELECT * FROM "prediction"."data"')
                    if row:
                        # Specify the CSV file path
                        csv_file_path = self.config.local_data_file

                        # Write named tuples to CSV file
                        with open(csv_file_path, 'w', newline='') as csv_file:
                            writer = csv.writer(csv_file)
                            
                            # Write header
                            writer.writerow(row[0]._fields)
                            
                            # Write data
                            for each in row:
                                writer.writerow(each)
                            logger.info(f"{self.config.local_data_file} download! from astra db with following info: \n {row[0]._fields}")
                    else:
                        logger.info(f"An Error occurred while connecting to db")
                    
                else:
                    logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            except Exception as e:
                logger.info(f"Delay for next attempt {e}")
                time.sleep(delay)
                delay *= 2
        else:
            logger.info(f"Failed connecting to astra/casandra db after {max_retries} attempt")


In [7]:
from titanicSpaceShip.constants import *

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                local_data_file=config.local_data_file,
                cloud_config_zipfile=config.cloud_config_zipfile,
                authentication_token=config.authentication_token
          )
          return data_ingestion_config

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    # data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-14 12:01:12,046: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-14 12:01:12,052: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-14 12:01:12,058: INFO: common: created directory at: artifacts]
[2024-01-14 12:01:12,060: INFO: common: created directory at: artifacts/data_ingestion]


[2024-01-14 12:01:12,656: WARNING: 681926982: Downgrading core protocol version from 66 to 65 for 43c9e6f8-a643-4970-8ad4-6fe67303cbd1-asia-south1.db.astra.datastax.com:29042:056a4a91-2687-448b-9873-6730a3416eea. To avoid this, it is best practice to explicitly set Cluster(protocol_version) to the version supported by your cluster. http://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Cluster.protocol_version]
[2024-01-14 12:01:12,788: WARNING: 681926982: Downgrading core protocol version from 65 to 5 for 43c9e6f8-a643-4970-8ad4-6fe67303cbd1-asia-south1.db.astra.datastax.com:29042:056a4a91-2687-448b-9873-6730a3416eea. To avoid this, it is best practice to explicitly set Cluster(protocol_version) to the version supported by your cluster. http://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Cluster.protocol_version]
[2024-01-14 12:01:12,960: ERROR: asyncorereactor: Closing connection <AsyncoreConnection(140285896216080) 43c9e

/tmp/ipykernel_69861/681926982.py:43: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  writer.writerow(row[0]._fields)


[2024-01-14 12:01:14,543: INFO: 681926982: artifacts/data_ingestion/data_spaceShip.csv download! from astra db with following info: 
 ('SNo', 'Age', 'CryoSleep', 'Destination', 'FoodCourt', 'HomePlanet', 'RoomService', 'ShoppingMall', 'Spa', 'Transported', 'VIP', 'VRDeck')]
[2024-01-14 12:01:14,544: INFO: 681926982: Failed connecting to astra/casandra db after 1 attempt]


/tmp/ipykernel_69861/681926982.py:48: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  logger.info(f"{self.config.local_data_file} download! from astra db with following info: \n {row[0]._fields}")
